In [1]:
%run ./01-basic.ipynb

# Intro

# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

## Create our QandA application

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [3]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [4]:
# create the vector store
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [5]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Coming up with test datapoints

In [6]:
data[10]

Document(page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10})

In [7]:
data[12]

Document(page_content=': 12\nname: CozyCloud Flannel Sheet Set\ndescription: Our Ultrasoft Flannel Bedding solid colored soft flannel sheet set is the perfect choice for those looking for year-round comfort. With its five-ounce weight and brushed cotton flannel construction, it will keep its bright colors wash after wash. It has been tested for harmful substances to the STANDARD 100 by OEKO-TEX® Certification (137 CITEVE) and is woven in Portugal exclusively for . This set includes one flat sheet, one fitted sheet and two cases; Twin has one case. \n\nSpecs:\nTwin: \nFlat: 102" x 72".\nFitted: 76" x 39" x 15" pocket depth.\nPillowcases (1): 32" x 20".\n\nFull: \nFlat: 102" x 86".\nFitted: 76" x 54" x 15" pocket depth.\nPillowcases (2): 32" x 20".\n\nQueen: \nFlat: 104" x 96".\nFitted: 80" x 60" x 15" pocket depth.\nPillowcases (2): 32" x 20".\n\nKing: \nFlat: 110', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 12})

### Hard-coded examples

In [8]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM Generated Examples

In [9]:
from langchain.evaluation.qa import QAGenerateChain


In [10]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [11]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

In [13]:
new_examples[0]

{'query': "What is the weight per pair of the Women's Campside Oxfords?",
 'answer': "The Women's Campside Oxfords have an approximate weight of 1 lb. 1 oz. per pair."}

### Combine Examples

In [14]:
examples += new_examples

In [15]:
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'The Cozy Comfort Pullover Set, Stripe has side pockets.'

In [16]:
qa.run(examples[3]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'The dimensions of the medium-sized Recycled Waterhog Dog Mat are 22.5" x 34.5".'

### Manual Evaluation

In [17]:
import langchain
langchain.debug = True

In [18]:
qa.run(examples[0]["query"])

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded h

'The Cozy Comfort Pullover Set, Stripe has side pockets on the pull-on pants.'

In [19]:
# Turn off the debugging mode
langchain.debug = False

## LLM-assisted evaluation

In [20]:
predictions = qa.apply(examples)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [21]:
from langchain.evaluation.qa import QAEvalChain

In [22]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [23]:
graded_outputs = eval_chain.evaluate(examples, predictions)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 75715ef6b06c0b42a32ddb8a785278da in your message.).


In [24]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: The Cozy Comfort Pullover Set, Stripe has side pockets.
Predicted Grade: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: CORRECT

Example 2:
Question: What is the weight per pair of the Women's Campside Oxfords?
Real Answer: The Women's Campside Oxfords have an approximate weight of 1 lb. 1 oz. per pair.
Predicted Answer: The weight per pair of the Women's Campside Oxfords is approximately 1 lb. 1 oz.
Predicted Grade: CORRECT

Example 3:
Question: What are the dimensions of the medium-sized Recycled Waterhog Dog Mat?
Real Answer: The dimensions of the medium-sized Recycled Waterhog Dog Mat are 22.5" x 34.5".
Predicted Answer: The dimensions of the medium-sized Recy

# Langchain plus 
- session information is stored.
- latency/performance is measured.
- nicely formatted debug-friendly.. 
- exampless can be added to the dataset
- analytics? 
